### 3.5.5 完整神经网络样例程序
可以分为三个步骤：
1. 定义神经网络参数，输入和输出节点；
2. 定义前向传播、损失函数和反向传播优化算法；
3. 生成会话，并在训练集上反复运行反向传播优化算法。

In [1]:
import tensorflow as tf
from numpy.random import RandomState

# 生成模拟数据集
rdm = RandomState(1)
X = rdm.rand(128, 2)
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

#### 1. 定义神经网络的参数，输入和输出节点。

In [2]:
BATCH_SIZE = 8  # 每一个batch中样本的数量
STEPS = 5000    # NN运行迭代的轮数

w1= tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_= tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

#### 2. 定义前向传播过程，损失函数及反向传播算法。
损失函数和反向传播第四章会有详述。

In [3]:
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
y = tf.sigmoid(y)

cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
                                + (1 - y_) * tf.log(tf.clip_by_value(1 - y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

#### 3. 创建一个会话来运行TensorFlow程序。

In [4]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 输出目前（未经训练）的参数取值。
    print(sess.run(w1))
    print(sess.run(w2))
    print("\n")
    
    # 训练模型。
    for i in range(STEPS):
        start = (i * BATCH_SIZE) % 128
        end = (i * BATCH_SIZE) % 128 + BATCH_SIZE
        sess.run([train_step, y, y_], feed_dict={x:X[start:end], y_:Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            print("After %d training step(s), cross entropy on all data is %g" % (i, total_cross_entropy))
    
    # 输出训练后的参数取值。
    print("\n")
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]


After 0 training step(s), cross entropy on all data is 1.89805
After 1000 training step(s), cross entropy on all data is 0.655075
After 2000 training step(s), cross entropy on all data is 0.626172
After 3000 training step(s), cross entropy on all data is 0.615096
After 4000 training step(s), cross entropy on all data is 0.610309


[[ 0.02476974  0.56948686  1.6921943 ]
 [-2.1977353  -0.23668927  1.1143897 ]]
[[-0.45544702]
 [ 0.49110925]
 [-0.98110336]]
